# Evaluating code-completion models on code samples

## load data
I will use files with code from my [repository](https://github.com/SzymonKozl/kierki) with c++ game server.

In [1]:
!git clone https://github.com/SzymonKozl/kierki

Cloning into 'kierki'...
remote: Enumerating objects: 806, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 806 (delta 166), reused 182 (delta 98), pack-reused 548 (from 1)
Receiving objects: 100% (806/806), 167.73 KiB | 3.11 MiB/s, done.
Resolving deltas: 100% (585/585), done.
